## DataFrame API Parquet - Query2

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sc = SparkSession \
    .builder \
    .appName("ex2") \
    .getOrCreate() 



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3469,application_1732639283265_3425,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'

df = spark.read.csv(data_path, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data2_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
df2 = spark.read.csv(data2_path,header = True, inferSchema = True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_combined = df.union(df2)
df_combined.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA ', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON']

In [5]:
# Αποθήκευση σε Parquet στο S3
group_name = '9'
s3_path = f"s3://groups-bucket-dblab-905418150721/group{group_name}/q2/"
df_combined.write.mode("overwrite").parquet(s3_path)

print(f"Data successfully written to {s3_path} in Parquet format.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data successfully written to s3://groups-bucket-dblab-905418150721/group9/q2/ in Parquet format.

In [6]:
import time
start_time = time.time()
# Φόρτωση των δεδομένων από το Parquet αρχείο
df_parquet = spark.read.parquet(s3_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_parquet.count()
df_parquet.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA : integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- 

In [8]:
df_parquet.select('AREA NAME').distinct().show() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|  AREA NAME|
+-----------+
|     Harbor|
|N Hollywood|
|   Van Nuys|
|West Valley|
|   Wilshire|
|    Topanga|
|   Foothill|
| Devonshire|
|    Pacific|
|    Central|
|    West LA|
|  Northeast|
|    Mission|
|  Southeast|
|    Olympic|
|  Southwest|
|    Rampart|
|77th Street|
|  Hollywood|
|     Newton|
+-----------+
only showing top 20 rows

In [9]:
df_parquet.select('Status','Status Desc').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------+
|Status| Status Desc|
+------+------------+
|    JA|  Juv Arrest|
|    AA|Adult Arrest|
|    JO|   Juv Other|
|    AO| Adult Other|
|    IC| Invest Cont|
|    TH|         UNK|
|    CC|         UNK|
|  NULL|         UNK|
|    19|         UNK|
|    13|         UNK|
+------+------------+

In [10]:
from pyspark.sql.functions import year, to_date

# Ensure legacy time parser is enabled (if required)
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df_parquet = df_parquet.withColumn('Year', year(to_date('Date Rptd', 'MM/dd/yyyy hh:mm:ss a')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql import functions as F

# Υπολογισμός συνολικών περιπτώσεων και κλειστών περιπτώσεων ανά έτος και περιοχή
aggregated_data = df_parquet.filter(
    df_parquet['Status'].isin('JA', 'AA', 'JO', 'AO')
).groupBy('Year', 'AREA NAME').agg(
    F.count('*').alias('closed_cases')
).join(
    df_parquet.groupBy('Year', 'AREA NAME').agg(
        F.count('*').alias('total_cases')
    ),
    on=['Year', 'AREA NAME']
)

# Υπολογισμός ποσοστού
result = aggregated_data.withColumn(
    'closed_case_rate',
    (F.col('closed_cases') / F.col('total_cases') * 100)
).select('Year', 'AREA NAME', 'closed_case_rate')

# Υπολογισμός κατάταξης
from pyspark.sql.window import Window
window_spec = Window.partitionBy('Year').orderBy(F.col('closed_case_rate').desc())

ranked_results = result.withColumn(
    'rank',
    F.rank().over(window_spec)
).filter(F.col('rank') <= 3)

# Συλλογή αποτελεσμάτων
final_results = ranked_results.collect()

# Εμφάνιση
print(f"{'year':<6} {'precinct':<20} {'closed_case_rate':<20} {'#':<5}")
print("-" * 60)
for row in final_results:
    print(f"{row['Year']:<6} {row['AREA NAME']:<20} {row['closed_case_rate']:<20.5f} {row['rank']:<5}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time parquet taken: {elapsed_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

year   precinct             closed_case_rate     #    
------------------------------------------------------------
2010   Rampart              32.94736             1    
2010   Olympic              31.96271             2    
2010   Harbor               29.63203             3    
2011   Olympic              35.21217             1    
2011   Rampart              32.51178             2    
2011   Harbor               28.65221             3    
2012   Olympic              34.41482             1    
2012   Rampart              32.94642             2    
2012   Harbor               29.81513             3    
2013   Olympic              33.52812             1    
2013   Rampart              32.08287             2    
2013   Harbor               29.16422             3    
2014   Van Nuys             31.80567             1    
2014   West Valley          31.31199             2    
2014   Mission              31.16279             3    
2015   Van Nuys             32.64135             1    
2015